In [1]:


!pip install pathway bokeh panel --quiet

import pathway as pw
import pandas as pd
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
import panel as pn

pn.extension()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:

df = pd.read_csv("/content/dataset.csv")
print("Column Names:\n", df.columns.tolist())

df.head()


Column Names:
 ['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime']


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [3]:

df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)

df.drop(['LastUpdatedDate', 'LastUpdatedTime', 'ID'], axis=1, inplace=True)

df = df.sort_values(by=['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)

df.head()


,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,Timestamp
0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,2016-10-04 07:59:00
1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,2016-10-04 08:25:00
2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,2016-10-04 08:59:00
3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,2016-10-04 09:32:00
4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,2016-10-04 09:59:00


In [4]:

BASE_PRICE = 10
ALPHA = 2.0


df['Price_Model1'] = np.nan


for lot, group in df.groupby('SystemCodeNumber'):
    price = BASE_PRICE
    indices = group.index.tolist()

    for i in indices:
        occupancy = df.loc[i, 'Occupancy']
        capacity = df.loc[i, 'Capacity']
        delta = ALPHA * (occupancy / capacity)


        df.loc[i, 'Price_Model1'] = price

        price = price + delta

df['Price_Model1'] = df['Price_Model1'].round(2)

df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'Capacity', 'Price_Model1']].head(10)

,SystemCodeNumber,Timestamp,Occupancy,Capacity,Price_Model1
0,BHMBCCMKT01,2016-10-04 07:59:00,61,577,10.00
1,BHMBCCMKT01,2016-10-04 08:25:00,64,577,10.21
2,BHMBCCMKT01,2016-10-04 08:59:00,80,577,10.43
3,BHMBCCMKT01,2016-10-04 09:32:00,107,577,10.71
4,BHMBCCMKT01,2016-10-04 09:59:00,150,577,11.08
5,BHMBCCMKT01,2016-10-04 10:26:00,177,577,11.60
6,BHMBCCMKT01,2016-10-04 10:59:00,219,577,12.21
7,BHMBCCMKT01,2016-10-04 11:25:00,247,577,12.97
8,BHMBCCMKT01,2016-10-04 11:59:00,259,577,13.83
9,BHMBCCMKT01,2016-10-04 12:29:00,266,577,14.73


In [5]:

vehicle_weights = {
    'car': 1.0,
    'bike': 0.5,
    'truck': 1.5,
    'cycle': 0.3
}


traffic_map = {
    'low': 0.2,
    'medium': 0.5,
    'average': 0.5,
    'high': 1.0
}


df['VehicleWeight'] = df['VehicleType'].map(vehicle_weights)
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)


In [8]:
BASE_PRICE = 10
alpha = 2.0
beta = 1.5
gamma = 1.0
delta = 2.0
epsilon = 1.0
lambda_factor = 0.2


In [9]:
df['RawDemand'] = (
    alpha * (df['Occupancy'] / df['Capacity']) +
    beta * df['QueueLength'] -
    gamma * df['TrafficLevel'] +
    delta * df['IsSpecialDay'] +
    epsilon * df['VehicleWeight']
)

df['NormalizedDemand'] = df.groupby('SystemCodeNumber')['RawDemand'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min() + 1e-6)
)

df['Price_Model2'] = BASE_PRICE * (1 + lambda_factor * df['NormalizedDemand'])
df['Price_Model2'] = df['Price_Model2'].clip(lower=0.5 * BASE_PRICE, upper=2.0 * BASE_PRICE)
df['Price_Model2'] = df['Price_Model2'].round(2)


In [11]:
df['NormalizedDemand'] = df.groupby('SystemCodeNumber')['RawDemand'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min() + 1e-6)
)

In [12]:

df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'QueueLength', 'VehicleType',
    'TrafficConditionNearby', 'IsSpecialDay', 'Price_Model2']].head(10)

,SystemCodeNumber,Timestamp,Occupancy,QueueLength,VehicleType,TrafficConditionNearby,IsSpecialDay,Price_Model2
0,BHMBCCMKT01,2016-10-04 07:59:00,61,1,car,low,0,10.22
1,BHMBCCMKT01,2016-10-04 08:25:00,64,1,car,low,0,10.23
2,BHMBCCMKT01,2016-10-04 08:59:00,80,2,car,low,0,10.39
3,BHMBCCMKT01,2016-10-04 09:32:00,107,2,car,low,0,10.40
4,BHMBCCMKT01,2016-10-04 09:59:00,150,2,bike,low,0,10.36
5,BHMBCCMKT01,2016-10-04 10:26:00,177,3,car,low,0,10.58
6,BHMBCCMKT01,2016-10-04 10:59:00,219,6,truck,high,0,11.02
7,BHMBCCMKT01,2016-10-04 11:25:00,247,5,car,average,0,10.88
8,BHMBCCMKT01,2016-10-04 11:59:00,259,5,cycle,average,0,10.81
9,BHMBCCMKT01,2016-10-04 12:29:00,266,8,bike,high,0,11.25


In [13]:
lambda_factor = 1.2
df['Price_Model2'] = BASE_PRICE * (1 + lambda_factor * df['NormalizedDemand'])
df['Price_Model2'] = df['Price_Model2'].clip(lower=0.5 * BASE_PRICE, upper=2.0 * BASE_PRICE)
df['Price_Model2'] = df['Price_Model2'].round(2)


df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'QueueLength', 'VehicleType', 'TrafficConditionNearby',
    'IsSpecialDay', 'Price_Model2']].head(10)

,SystemCodeNumber,Timestamp,Occupancy,QueueLength,VehicleType,TrafficConditionNearby,IsSpecialDay,Price_Model2
0,BHMBCCMKT01,2016-10-04 07:59:00,61,1,car,low,0,11.35
1,BHMBCCMKT01,2016-10-04 08:25:00,64,1,car,low,0,11.35
2,BHMBCCMKT01,2016-10-04 08:59:00,80,2,car,low,0,12.32
3,BHMBCCMKT01,2016-10-04 09:32:00,107,2,car,low,0,12.37
4,BHMBCCMKT01,2016-10-04 09:59:00,150,2,bike,low,0,12.16
5,BHMBCCMKT01,2016-10-04 10:26:00,177,3,car,low,0,13.45
6,BHMBCCMKT01,2016-10-04 10:59:00,219,6,truck,high,0,16.15
7,BHMBCCMKT01,2016-10-04 11:25:00,247,5,car,average,0,15.28
8,BHMBCCMKT01,2016-10-04 11:59:00,259,5,cycle,average,0,14.87
9,BHMBCCMKT01,2016-10-04 12:29:00,266,8,bike,high,0,17.49


In [14]:
from sklearn.neighbors import NearestNeighbors

df_competitor = df.copy()
df_competitor['Latitude'] = df_competitor['Latitude'].astype(float)
df_competitor['Longitude'] = df_competitor['Longitude'].astype(float)

lots = df_competitor.groupby('SystemCodeNumber')[['Latitude', 'Longitude']].mean().reset_index()
neighbors_model = NearestNeighbors(n_neighbors=3, metric='euclidean')
neighbors_model.fit(lots[['Latitude', 'Longitude']])
distances, indices = neighbors_model.kneighbors(lots[['Latitude', 'Longitude']])

neighbor_map = dict()
for i, lot in enumerate(lots['SystemCodeNumber']):
    neighbor_lots = lots.iloc[indices[i]]['SystemCodeNumber'].tolist()
    neighbor_lots.remove(lot)  # remove itself
    neighbor_map[lot] = neighbor_lots

def compute_model3_price(row):
    lot = row['SystemCodeNumber']
    timestamp = row['Timestamp']
    neighbors = neighbor_map.get(lot, [])
    competitors = df_competitor[(df_competitor['SystemCodeNumber'].isin(neighbors)) &
                                (df_competitor['Timestamp'] == timestamp)]

    if competitors.empty:
        return row['Price_Model2']

    avg_competitor_price = competitors['Price_Model2'].mean()

    if row['Price_Model2'] < avg_competitor_price:
        return min(row['Price_Model2'] * 1.05, 2.0 * BASE_PRICE)
    elif row['Price_Model2'] > avg_competitor_price:
        return max(row['Price_Model2'] * 0.95, 0.5 * BASE_PRICE)
    else:
        return row['Price_Model2']


df['Price_Model3'] = df.apply(compute_model3_price, axis=1)
df['Price_Model3'] = df['Price_Model3'].round(2)

In [15]:
df[['SystemCodeNumber', 'Timestamp', 'Price_Model1', 'Price_Model2', 'Price_Model3']].head(10)


,SystemCodeNumber,Timestamp,Price_Model1,Price_Model2,Price_Model3
0,BHMBCCMKT01,2016-10-04 07:59:00,10.00,11.35,11.92
1,BHMBCCMKT01,2016-10-04 08:25:00,10.21,11.35,10.78
2,BHMBCCMKT01,2016-10-04 08:59:00,10.43,12.32,11.70
3,BHMBCCMKT01,2016-10-04 09:32:00,10.71,12.37,11.75
4,BHMBCCMKT01,2016-10-04 09:59:00,11.08,12.16,11.55
5,BHMBCCMKT01,2016-10-04 10:26:00,11.60,13.45,12.78
6,BHMBCCMKT01,2016-10-04 10:59:00,12.21,16.15,15.34
7,BHMBCCMKT01,2016-10-04 11:25:00,12.97,15.28,14.52
8,BHMBCCMKT01,2016-10-04 11:59:00,13.83,14.87,14.13
9,BHMBCCMKT01,2016-10-04 12:29:00,14.73,17.49,18.36


In [23]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Legend
from bokeh.palettes import Category10
from bokeh.layouts import column

output_notebook()


df['Timestamp'] = pd.to_datetime(df['Timestamp'])

p = figure(x_axis_type="datetime", width=800, height=400, title="Parking Price Trends (Model 1 vs 2 vs 3)")
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Price'

colors = Category10[3]
line1 = p.line(df['Timestamp'], df['Price_Model1'], line_color=colors[0], line_width=2, legend_label="Model 1: Occupancy-Based")
line2 = p.line(df['Timestamp'], df['Price_Model2'], line_color=colors[1], line_width=2, legend_label="Model 2: Demand-Based")
p.line(df['Timestamp'], df['Price_Model3'], line_color="#2ca02c", line_width=2, legend_label="Model 3: Competitor-Based")

p.legend.location = "top_left"
show(p)


In [18]:
df.to_csv("dynamic_pricing_output.csv", index=False)
print("File saved successfully as dynamic_pricing_output.csv")

File saved 
